### LIBRARY

In [1]:
import pandas as pd
import requests
import random
import time
import math

### SET UP THE ENVIRONMENT

In [2]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0",
    "Accept-Language": 'en-US,en;q=0.9',
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Referer": "https://tiki.vn/",
    "From": "",
    "af-ac-enc-dat": "",
    "x-api-source": "pc"
}

### EXTRACT GROUP DATA

In [3]:
URL = "https://api.tiki.vn/raiden/v2/menu-config?platform=desktop"

In [4]:
response = requests.get(URL, headers=HEADERS)

if response.status_code == 200:
    data = response.json()
    print(f"Success to fetch {len(data["menu_block"]["items"])} groups.")
else:
    print("Failed to fetch data:", response.status_code)

Success to fetch 26 groups.


### EXTRACT GROUP ID

In [5]:
group = data["menu_block"]["items"]
group_list = []

for group in group:
    text = group["text"]
    link = group["link"]
    # Extract category ID from the link
    group_id = link.split("/")[-1][1:]  
    group_list.append([group_id, text])

group = pd.DataFrame(group_list, columns=["group_id", "group_name"])
group.head()

,group_id,group_name
0,8322,Nhà Sách Tiki
1,1883,Nhà Cửa - Đời Sống
2,1789,Điện Thoại - Máy Tính Bảng
3,2549,Đồ Chơi - Mẹ & Bé
4,1815,Thiết Bị Số - Phụ Kiện Số


### EXTRACT CATEGORY ID

In [6]:
category = group

In [7]:
parent_category_list = []
child_category_list = []

for group_id in category["group_id"]:
    parent_url = f"https://tiki.vn/api/v2/categories?parent_id={group_id}"
    parent_response = requests.get(parent_url, headers=HEADERS)
    if parent_response.status_code == 200:
        parent_data = parent_response.json()
        if not parent_data["data"]:
            parent_category_list.append([group_id, category[category["group_id"] == group_id]["Name"].item(), None, None])
        else:
            for parent_category in parent_data["data"]:
                parent_id = parent_category["id"]
                parent_name = parent_category["name"]
                # Get group name
                group_name = category[category["group_id"] == group_id]["group_name"].item()
                parent_category_list.append([group_id, group_name, parent_id, parent_name])
                # Now fetch child categories
                child_url = f"https://tiki.vn/api/v2/categories?parent_id={parent_id}"
                child_response = requests.get(child_url, headers=HEADERS)
                if child_response.status_code == 200:
                    child_data = child_response.json()
                    if not child_data["data"]:
                        child_category_list.append([group_id, group_name, parent_id, parent_name, None, None])
                    else:
                        for child_category in child_data["data"]:
                            child_id = child_category["id"]
                            child_name = child_category["name"]
                            child_count = child_category["product_count"]
                            child_category_list.append([group_id, group_name, parent_id, parent_name, child_id, child_name, child_count])
    else:
        print(f"Failed to fetch category data for group ID: {group_id}")

print(f"Success to fetch {len(parent_category_list)} categories.")
print(f"Success to fetch {len(child_category_list)} sub-categories.")

Success to fetch 230 categories.
Success to fetch 1372 sub-categories.


In [8]:
category = pd.DataFrame(child_category_list, columns=["group_id", "group_name", "parent_id", "parent_name", "child_id", "child_name", "product_count"])
category

,group_id,group_name,parent_id,parent_name,child_id,child_name,product_count
0,8322,Nhà Sách Tiki,320,English Books,623.0,Art & Photography,576.0
1,8322,Nhà Sách Tiki,320,English Books,27.0,Biographies & Memoirs,314.0
2,8322,Nhà Sách Tiki,320,English Books,4.0,Business & Economics,654.0
3,8322,Nhà Sách Tiki,320,English Books,614.0,How-to - Self Help,662.0
4,8322,Nhà Sách Tiki,320,English Books,7.0,Children's Books,7592.0
...,...,...,...,...,...,...,...
1367,15078,Chăm sóc nhà cửa,4441,Diệt côn trùng,68132.0,Diệt côn trùng dạng bột,168.0
1368,15078,Chăm sóc nhà cửa,4441,Diệt côn trùng,68133.0,Diệt côn trùng dạng nhang đốt,24.0
1369,15078,Chăm sóc nhà cửa,4441,Diệt côn trùng,68137.0,Diệt côn trùng dạng nước và tinh dầu,380.0
1370,15078,Chăm sóc nhà cửa,4441,Diệt côn trùng,68138.0,Diệt côn trùng dạng sáp và viên nén,109.0
